In [2]:
# Create Spectrograms from Urbansound8k using TorchAudio

# Performance was observed to be highest after a computer restart

# Uses Tutorial: 
# https://bagustris.wordpress.com/2022/04/22/basic-audio-manipulation-with-torchaudio/

# os for file operations
import os

# torch audio to do gpu spectrograms
# librosa to do regular spectrograms
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import librosa
import librosa.display

# numpy for calculations
import numpy as np

# matplotlib for graphs
import matplotlib.pyplot as plt
%matplotlib inline

#skimage io to save spectrogram
#import skimage.io

# Timer for the process
import time

In [3]:
# # Pathlib Test
# from pathlib import Path

# path_p = Path('N:\\# GMU 2022 ML Model\\UrbanSound8K\\')

# print(path_p)

In [4]:
# # CSV Import and Processing
# # Using Tutorial: audio-deep-learning-made-simple-sound
# # https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5


# # ----------------------------
# # Prepare training data from Metadata file
# # ----------------------------

# import pandas as pd
# from pathlib import Path

# download_path = Path('N:\\# GMU 2022 ML Model\\UrbanSound8K\\')

# # Read metadata file
# #metadata_file = 'N:\\# GMU 2022 ML Model\\UrbanSound8K\\metadata\\UrbanSound8K.csv'

# metadata_file = download_path/'metadata'/'UrbanSound8K.csv'
# df = pd.read_csv(metadata_file)
# df.head()

# # Construct file path by concatenating fold and file name
# df['relative_path'] = '/fold' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str)

# # Take relevant columns
# #df = df[['relative_path', 'classID']]
# df.head()

In [5]:
#'N:\\# GMU 2022 ML Model\\UrbanSound8K\\metadata\\UrbanSound8K.csv'

In [6]:
#N:\# GMU 2022 ML Model\UrbanSound8K\metadata

In [7]:
# CUDA Test
# geeksforgeeks
# https://www.geeksforgeeks.org/how-to-set-up-and-run-cuda-operations-in-pytorch/
import torch

import tensorflow as tf

from tensorflow.python.client import device_lib

def cuda_test():
    #
    print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
    print(f"CUDA version: {torch.version.cuda}")

    # Storing ID of current CUDA device
    print("\n\n\n")
    cuda_id = torch.cuda.current_device()
    print(f"ID of current CUDA device: {torch.cuda.current_device()}")
    print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")
    
    #
    print("\n\n\n")
    print("Tensorflow GPU Test")
    num_devices = tf.config.list_physical_devices('GPU')

    print("Num GPUs Available: ", len(num_devices))
    print(device_lib.list_local_devices())
    
    #
    print("\n\n\n")
    if torch.cuda.is_available():
        print('We have a GPU!')
    else:
        print('Sorry, CPU only.')

In [8]:
cuda_test()

Is CUDA supported by this system? True
CUDA version: 11.6




ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 2070 SUPER




Tensorflow GPU Test
Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5259158052348277239
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6254755840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8228446571789186485
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]




We have a GPU!


In [9]:
# Using tutorials and API:
# Pytorch audio feature extraction tutorial
# https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html?highlight=mel+spectrogram
# Code from pytorch-intro-with-torchaudio
# https://blog.deepgram.com/pytorch-intro-with-torchaudio/

def plot_waveform(waveform, sample_rate):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("waveform")
    plt.show(block=False)

# From pytorch documentation tutorial
# https://pytorch.org/audio/main/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", cmap='magma')
    fig.colorbar(im, ax=axs)
#    plt.set_cmap('magma')
    plt.show(block=False)

In [10]:
# Using Code from  Extracting Mel Spectrograms with Pytorch and Torchaudio 
# https://www.youtube.com/watch?v=lhF_RVa7DLE
# https://github.com/musikalkemist/pytorchforaudio/blob/main/05%20Extracting%20Mel%20spectrograms/urbansounddataset.py
# Resample rate conversion
def resample_if_necessary(signal, sr_file, sr):
    if sr_file != sr:
        resampler = torchaudio.transforms.Resample(sr_file, sr)
        signal = resampler(signal)
        print("resampled")
    return signal

# Stereo to mono conversion
def mix_down_if_necessary(signal):
    if signal.shape[0] > 1:
        signal = torch.mean(signal, dim=0, keepdim=True)
        print("downmixed")
    return signal

# # Segments the length of an input signal to segments of a target size if the signal is longer than the target size
# def _segment_if_necessary(signal, target_num_samples):
#     if signal.shape[1] > target_num_samples:
#         signal = signal[:, :self.num_samples]
    
#     # Returns an array of the 
#     return signal

# Trims the length of an input signal to a target size if it exceeds the length limit
def _cut_if_necessary(signal, target_num_samples):
    if signal.shape[1] > target_num_samples:
        signal = signal[:, :self.num_samples]
    
    # Returns an array of the 
    return signal

# Tutorial Implementation
# # Adds zero padding to a signal to increase its duration to a target size.  Used for signals shorter than target size
# def _right_pad_if_necessary(signal, target_num_samples):
#     length_signal = signal.shape[1]
#     if length_signal < self.num_samples:
#         num_missing_samples = self.num_samples - length_signal
#         last_dim_padding = (0, num_missing_samples)
#         signal = torch.nn.functional.pad(signal, last_dim_padding)
#     return signal

# Context Specific Implementation
# Context = Signal must be composed of an even number of 1 second clips or else right pad with zeros
# to make this true
def right_pad_if_necessary(signal, chunk_size_needed):
    #print(len(signal[0])
    signal_padded = signal
    if (len(signal[0])%chunk_size_needed != 0):
        num_missing_samples = chunk_size_needed - len(signal[0])%chunk_size_needed
        signal_padded = torch.nn.functional.pad(input = signal, pad = (0, num_missing_samples))
#         print(f"len(signal) = {len(signal)}")
#         print(f"chunk_size = {chunk_size_needed}")
#         print(f"{chunk_size_needed} - {len(signal)} = num samples {num_missing_samples}")
    
    return signal_padded

# Pads an input signal less than chunk_size_needed and truncates the ends of signals that are more than 
# one second but not a exactly  multiple of 1 second.
def pad_or_shorten(signal, chunk_size_needed):
    
    # Pad if example is less than 1 second
    if (len(signal[0]) < chunk_size_needed):
        num_missing_samples = chunk_size_needed - len(signal[0])%chunk_size_needed
        signal_processed = torch.nn.functional.pad(input = signal, pad = (0, num_missing_samples))
    elif (len(signal[0]) > chunk_size_needed):
        signal_processed[0] = signal[0][:(chunk_size_needed*(len(signal[0])//chunk_size_needed))]
        print(f"Signal Loss: {len(signal_processed[0])-len(signal[0])}")

In [11]:

def create_mel_specs_split_torch(dir_name, folder_num, num_spect=10, override= False):
    # Find audio files in target directory
    target_dir = os.path.join(dir_name,f"fold{folder_num}")
    filenames = os.listdir(target_dir)

    # Print number of files and information about first 2 elements
    print(len(filenames))
    print(filenames[0])
    print(filenames[1][-4:])

    # Count cycles and starting time
    counter_clips = 0
    counter_spectrograms = 0
    time_start = time.time()
    
    # Parameters
    frame_size = 512
    hop_length = 256
    sr = 16000

    # Create torch transformer
    torch_transformer_tutorial = T.MelSpectrogram(
    sample_rate=sr,
    n_fft=frame_size,
    win_length=frame_size,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm="slaney",
    onesided=True,
    n_mels=128,
    mel_scale="htk",)
    
    print(f'\n\nEntering Loop, Time is: {time_start}')
    for elt in filenames:
        
        # Not a .wav file means skip
        if elt[-4:] != '.wav':
            continue;
        
        # Soft lock for large run size, only calculates given number of audio files.
        elif (override == False and counter_clips >= num_spect):
            break;
            
        else:
            output_filename = elt.split('.')[0] + '-melspec-torch'
            print(f'output filename is {output_filename}')
            # Using tutorial: How to Create & Understand Mel-Spectrograms
            # https://importchris.medium.com/how-to-create-understand-mel-spectrograms-ff7634991056

            # Try for sample rate target of 16000 Hz
            # Frame size = 512 because 1 / 16000 * 512 = .031 seconds or 30ms window
            # *This was just eyeballed from a small sample of siren patterns sadly
            # Hop_length 256 for 50% frame overlap (this is a hyperparameter other values should be tried)

            frame_size = 512
            hop_length = 256
            sr = 16000
            
            url = os.path.join(target_dir, elt)

            print(url)
            print(f'clip counter is: {counter_clips}')
            
            signal, sr_file = torchaudio.load(url)
            
            signal = mix_down_if_necessary(signal)
            signal = resample_if_necessary(signal, sr_file, sr)
            signal = right_pad_if_necessary(signal, sr)


            # Split signal into 1 second parts
            signal_arr = torch.split(signal, 16000, 1)
            
            for elt_i in range(len(signal_arr)):
                # Construct mel spectrogram with torchaudio GPU
                mel_spectrogram = torch_transformer_tutorial(signal_arr[elt_i])[0]

                # Librosa is still used because it was written somewhere that GPU normalization is not as good idk
                mel_spectrogram = librosa.power_to_db(mel_spectrogram)
          
                # Test code ignore
                # plt.imshow(librosa.power_to_db(mel_spectrogram), origin="lower", aspect="auto")

                # Imsave using plt, origin lower is important or image will be inverted
                save_path = os.path.join(target_dir, f"fold{folder_num}_spectrograms-torch-1sec")
                save_path = os.path.join(save_path, f"{output_filename}-{elt_i}.png")

                # print(f"Savepath is:{save_path}")
                # Origin lower is critical or image will be inverted
                plt.imsave(save_path, mel_spectrogram, origin = 'lower', cmap='magma')
                
                # Increment overall spectrogram counter
                counter_spectrograms += 1
            
            # Implement clip counter
            counter_clips += 1
            # Every 500 files print time update
            if counter_clips != 0 and counter_clips%500 == 0:
                t_elapsed = time.time() - time_start
                print(f"\n\n\n-=-=-= Time at {counter_clips}: {t_elapsed}\n\n\n")
            
            
    # Calculate time end and print performance benchmarks.
    time_end = time.time()
    print(f"Num Clips Processed: {counter_clips} , Num Spectrograms Created: {counter_spectrograms}")
    print(f"Task Ended. t_start = {time_start}. t_end = {time_end}")
    diff = time_end - time_start
    print(f'Time Elapsed: {diff} seconds')
    
    # Return tuple of time start and end
    return (time_start,time_end)

In [12]:
# def create_mel_specs_torch(dir_name, folder_num, num_spect=10, override= False):
#     # Find audio files in target directory
#     target_dir = os.path.join(dir_name,f"fold{folder_num}")
#     filenames = os.listdir(target_dir)

#     # Print number of files and information about first 2 elements
#     print(len(filenames))
#     print(filenames[0])
#     print(filenames[1][-4:])

#     # Count cycles and starting time
#     counter = 0
#     time_start = time.time()
    
#     # Parameters
#     frame_size = 512
#     hop_length = 256
#     sr = 16000
    
    
#     # Create torch transformer
#     torch_transformer_tutorial = T.MelSpectrogram(
#     sample_rate=sr,
#     n_fft=frame_size,
#     win_length=frame_size,
#     hop_length=hop_length,
#     center=True,
#     pad_mode="reflect",
#     power=2.0,
#     norm="slaney",
#     onesided=True,
#     n_mels=128,
#     mel_scale="htk",)
    
#     # Start loop
#     print(f'\n\nEntering Loop, Time is: {time_start}')
#     for elt in filenames:
        
#         # Not a .wav file means skip
#         if elt[-4:] != '.wav':
#             continue;
        
#         # Soft lock for large run size, only calculates given number of audio files.
#         elif (override == False and counter >= num_spect):
#             break;
            
#         else:
#             output_filename = elt.split('.')[0] + '-melspec-torch.png'
#             print(f'output filename is {output_filename}')
#             # Using tutorial: How to Create & Understand Mel-Spectrograms
#             # https://importchris.medium.com/how-to-create-understand-mel-spectrograms-ff7634991056

#             # Try for sample rate target of 16000 Hz
#             # Frame size = 512 because 1 / 16000 * 512 = .031 seconds or 30ms window
#             # *This was just eyeballed from a small sample of siren patterns sadly
#             # Hop_length 256 for 50% frame overlap (this is a hyperparameter other values should be tried)

#             frame_size = 512
#             hop_length = 256
#             sr = 16000
            
#             url = os.path.join(target_dir, elt)

#             print(url)
#             print(f'counter is: {counter}')
            
#             signal, sr_file = torchaudio.load(url)
            
#             signal = mix_down_if_necessary(signal)
#             signal = resample_if_necessary(signal, sr_file, sr)
            
#             # Moved out of loop
# #             torch_transformer_tutorial = T.MelSpectrogram(
# #                 sample_rate=sr,
# #                 n_fft=n_fft,
# #                 win_length=win_length,
# #                 hop_length=hop_length,
# #                 center=True,
# #                 pad_mode="reflect",
# #                 power=2.0,
# #                 norm="slaney",
# #                 onesided=True,
# #                 n_mels=96,
# #                 mel_scale="htk",
# #             )

#             # Construct mel spectrogram with torchaudio GPU
#             mel_spectrogram = torch_transformer_tutorial(signal)[0]
            
#             # Librosa is still used because it was written somewhere that GPU normalization is not as good idk
#             mel_spectrogram = librosa.power_to_db(mel_spectrogram)
          
#             # Test code ignore
#             # plt.imshow(librosa.power_to_db(mel_spectrogram), origin="lower", aspect="auto")

#             # Imsave using plt, origin lower is important or image will be inverted
#             save_path = os.path.join(target_dir, f"fold{folder_num}_spectrograms-torch-1sec")
#             save_path = os.path.join(save_path, output_filename)
            
#             # print(f"Savepath is:{save_path}")
#             # Origin lower is critical or image will be inverted
#             plt.imsave(save_path, mel_spectrogram, origin = 'lower', cmap='magma')
            
#             # Every 500 files print time update
#             if counter != 0 and counter%500 == 0:
#                 t_elapsed = time.time() - time_start
#                 print(f"\n\n\n-=-=-= Time at {counter}: {t_elapsed}\n\n\n")
#             counter += 1
            
#     # Calculate time end and print performance benchmarks.
#     time_end = time.time()
#     print(f"Task Ended. t_start = {time_start}. t_end = {time_end}")
#     diff = time_end - time_start
#     print(f'Time Elapsed: {diff} seconds')
    
#     # Return tuple of time start and end
#     return (time_start,time_end)

In [13]:
# Create folders for spectrograms

# This is the directory of your audio folder in the urbansound8k dataset
dir_name = 'N:\\# GMU 2022 ML Model\\UrbanSound8K\\audio'
       
for i in range(1,11,1):
    path = os.path.join(dir_name, f"fold{i}")
    path = os.path.join(path,f"fold{i}_spectrograms-torch-1sec")
    if os.path.isdir(path) == False:
        os.mkdir(path)
        print(f"Created Folder {i}.")
    else:
        print(f"Folder fold{i}_spectrograms-torch-1sec already exists.")

Folder fold1_spectrograms-torch-1sec already exists.
Folder fold2_spectrograms-torch-1sec already exists.
Folder fold3_spectrograms-torch-1sec already exists.
Folder fold4_spectrograms-torch-1sec already exists.
Folder fold5_spectrograms-torch-1sec already exists.
Folder fold6_spectrograms-torch-1sec already exists.
Folder fold7_spectrograms-torch-1sec already exists.
Folder fold8_spectrograms-torch-1sec already exists.
Folder fold9_spectrograms-torch-1sec already exists.
Folder fold10_spectrograms-torch-1sec already exists.


In [5]:
# Trial run for 10 spectrograms:
dir_name = 'N:\\# GMU 2022 ML Model\\UrbanSound8K\\audio'
folder_num = 1
create_mel_specs_split_torch(dir_name, folder_num, num_spect=10)

882
.DS_Store
.wav


Entering Loop, Time is: 1676370891.1078432
output filename is 101415-3-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-2.wav
clip counter is: 0
resampled
output filename is 101415-3-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-3.wav
clip counter is: 1
resampled
output filename is 101415-3-0-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-8.wav
clip counter is: 2
resampled
output filename is 102106-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102106-3-0-0.wav
clip counter is: 3
downmixed
resampled
output filename is 102305-6-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102305-6-0-0.wav
clip counter is: 4
downmixed
resampled
output filename is 102842-3-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102842-3-0-1.wav
clip counter is: 5
downmixed
resampled
output filename is 102842-3-1-0-melspec-torch
N:\# GMU 2022 ML Mod

C:\Users\TCX-14\anaconda3\lib\site-packages\torchaudio\functional\functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


(1676370891.1078432, 1676370891.2705636)

In [67]:
# Test run for folder 1
dir_name = 'N:\\# GMU 2022 ML Model\\UrbanSound8K\\audio'
folder_num = 1

# Override must be True for the system to calculate more than the default limit of 10 spectrograms
create_mel_specs_split_torch(dir_name, folder_num, override = False)

881
.DS_Store
.wav


Entering Loop, Time is: 1676171504.227165
output filename is 101415-3-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-2.wav
clip counter is: 0
resampled
output filename is 101415-3-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-3.wav
clip counter is: 1
resampled
output filename is 101415-3-0-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\101415-3-0-8.wav
clip counter is: 2
resampled
output filename is 102106-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102106-3-0-0.wav
clip counter is: 3
downmixed
resampled
output filename is 102305-6-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102305-6-0-0.wav
clip counter is: 4
downmixed
resampled
output filename is 102842-3-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\102842-3-0-1.wav
clip counter is: 5
downmixed
resampled
output filename is 102842-3-1-0-melspec-torch
N:\# GMU 2022 ML Mode

downmixed
resampled
output filename is 105415-2-0-4-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\105415-2-0-4.wav
clip counter is: 56
downmixed
resampled
output filename is 105415-2-0-6-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\105415-2-0-6.wav
clip counter is: 57
downmixed
resampled
output filename is 105415-2-0-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\105415-2-0-8.wav
clip counter is: 58
downmixed
resampled
output filename is 106905-8-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\106905-8-0-0.wav
clip counter is: 59
downmixed
resampled
output filename is 106905-8-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\106905-8-0-1.wav
clip counter is: 60
downmixed
resampled
output filename is 106905-8-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\106905-8-0-2.wav
clip counter is: 61
downmixed
resampled
output filename is 106905-8-0-3-melspec-torch
N:\# GMU 2022 ML Model\Urban

output filename is 118279-8-0-9-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\118279-8-0-9.wav
clip counter is: 111
downmixed
resampled
output filename is 118963-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\118963-3-0-0.wav
clip counter is: 112
downmixed
resampled
output filename is 119455-5-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\119455-5-0-0.wav
clip counter is: 113
downmixed
resampled
output filename is 119455-5-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\119455-5-0-1.wav
clip counter is: 114
downmixed
resampled
output filename is 119455-5-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\119455-5-0-2.wav
clip counter is: 115
downmixed
resampled
output filename is 119455-5-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\119455-5-0-3.wav
clip counter is: 116
downmixed
resampled
output filename is 119455-5-0-4-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\

downmixed
resampled
output filename is 135776-2-0-32-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-32.wav
clip counter is: 165
downmixed
resampled
output filename is 135776-2-0-37-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-37.wav
clip counter is: 166
downmixed
resampled
output filename is 135776-2-0-40-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-40.wav
clip counter is: 167
downmixed
resampled
output filename is 135776-2-0-49-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-49.wav
clip counter is: 168
downmixed
resampled
output filename is 135776-2-0-50-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-50.wav
clip counter is: 169
downmixed
resampled
output filename is 135776-2-0-65-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\135776-2-0-65.wav
clip counter is: 170
downmixed
resampled
output filename is 135776-2-0-85-melspec-torch
N:\# GMU 

output filename is 146186-5-0-9-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\146186-5-0-9.wav
clip counter is: 220
downmixed
resampled
output filename is 147764-4-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\147764-4-0-0.wav
clip counter is: 221
downmixed
resampled
output filename is 147764-4-1-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\147764-4-1-0.wav
clip counter is: 222
downmixed
resampled
output filename is 147764-4-2-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\147764-4-2-0.wav
clip counter is: 223
downmixed
resampled
output filename is 147764-4-3-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\147764-4-3-0.wav
clip counter is: 224
downmixed
resampled
output filename is 147764-4-4-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\147764-4-4-0.wav
clip counter is: 225
downmixed
resampled
output filename is 147764-4-4-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\

...

resampled
output filename is 46669-4-0-24-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-24.wav
clip counter is: 676
downmixed
resampled
output filename is 46669-4-0-26-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-26.wav
clip counter is: 677
downmixed
resampled
output filename is 46669-4-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-3.wav
clip counter is: 678
downmixed
resampled
output filename is 46669-4-0-30-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-30.wav
clip counter is: 679
downmixed
resampled
output filename is 46669-4-0-31-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-31.wav
clip counter is: 680
downmixed
resampled
output filename is 46669-4-0-32-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\46669-4-0-32.wav
clip counter is: 681
downmixed
resampled
output filename is 46669-4-0-33-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8

output filename is 57320-0-0-24-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-24.wav
clip counter is: 731
downmixed
resampled
output filename is 57320-0-0-27-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-27.wav
clip counter is: 732
downmixed
resampled
output filename is 57320-0-0-31-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-31.wav
clip counter is: 733
downmixed
resampled
output filename is 57320-0-0-32-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-32.wav
clip counter is: 734
downmixed
resampled
output filename is 57320-0-0-33-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-33.wav
clip counter is: 735
downmixed
resampled
output filename is 57320-0-0-34-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\57320-0-0-34.wav
clip counter is: 736
downmixed
resampled
output filename is 57320-0-0-36-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\

output filename is 73277-9-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-0.wav
clip counter is: 789
downmixed
resampled
output filename is 73277-9-0-16-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-16.wav
clip counter is: 790
downmixed
resampled
output filename is 73277-9-0-17-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-17.wav
clip counter is: 791
downmixed
resampled
output filename is 73277-9-0-19-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-19.wav
clip counter is: 792
downmixed
resampled
output filename is 73277-9-0-24-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-24.wav
clip counter is: 793
downmixed
resampled
output filename is 73277-9-0-28-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\73277-9-0-28.wav
clip counter is: 794
downmixed
resampled
output filename is 7383-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold

resampled
output filename is 97317-2-0-25-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\97317-2-0-25.wav
clip counter is: 851
downmixed
resampled
output filename is 97317-2-0-28-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\97317-2-0-28.wav
clip counter is: 852
downmixed
resampled
output filename is 97317-2-0-34-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\97317-2-0-34.wav
clip counter is: 853
downmixed
resampled
output filename is 97317-2-0-36-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\97317-2-0-36.wav
clip counter is: 854
downmixed
resampled
output filename is 97317-2-0-5-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\97317-2-0-5.wav
clip counter is: 855
downmixed
resampled
output filename is 98223-7-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold1\98223-7-0-0.wav
clip counter is: 856
downmixed
resampled
output filename is 98223-7-1-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\a

(1676171504.227165, 1676171547.7905493)

In [39]:
# Check to see if the spectrograms are in the folder

In [ ]:
# Test run for folder 2
dir_name = 'N:\\# GMU 2022 ML Model\\UrbanSound8K\\audio'
folder_num = 2

create_mel_specs_split_torch(dir_name, folder_num, override = False)

In [68]:
# Construct all spectrograms in urbansound8k audio folder
time_array = []
for fold_num in range(2,11,1):
    time_array.append(create_mel_specs_split_torch(dir_name, fold_num, override = True))

# Print performance metrics
sum_time_elapsed = 0

for elt_i in range(len(time_array)):
    time_elapsed = time_array[elt_i][1] - time_array[elt_i][0]
    
    sum_time_elapsed += time_elapsed
    
    print(f'Fold {elt_i}: t_start = {time_array[elt_i][0]} , t_end = {time_array[elt_i][1]}, time elapsed: {time_elapsed} seconds')

print(f"Sum time elapsed: {sum_time_elapsed} seconds")

895
.DS_Store
.wav


Entering Loop, Time is: 1676171567.0489209
output filename is 100652-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\100652-3-0-0.wav
clip counter is: 0
downmixed
resampled
output filename is 100652-3-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\100652-3-0-1.wav
clip counter is: 1
downmixed
resampled
output filename is 100652-3-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\100652-3-0-2.wav
clip counter is: 2
downmixed
resampled
output filename is 100652-3-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\100652-3-0-3.wav
clip counter is: 3
downmixed
resampled
output filename is 102104-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\102104-3-0-0.wav
clip counter is: 4
downmixed
resampled
output filename is 102858-5-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\102858-5-0-0.wav
clip counter is: 5
downmixed
resampled
output filename is 102858-5-0-1-mels

resampled
output filename is 108187-3-2-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\108187-3-2-0.wav
clip counter is: 56
downmixed
resampled
output filename is 108187-3-3-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\108187-3-3-0.wav
clip counter is: 57
downmixed
resampled
output filename is 108187-3-4-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\108187-3-4-0.wav
clip counter is: 58
downmixed
resampled
output filename is 108187-3-6-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\108187-3-6-0.wav
clip counter is: 59
downmixed
resampled
output filename is 109703-2-0-134-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\109703-2-0-134.wav
clip counter is: 60
downmixed
resampled
output filename is 109703-2-0-14-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\109703-2-0-14.wav
clip counter is: 61
downmixed
resampled
output filename is 109703-2-0-153-melspec-torch
N:\# GMU 2022 ML Model\UrbanSo

resampled
output filename is 123688-8-2-4-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\123688-8-2-4.wav
clip counter is: 113
downmixed
resampled
output filename is 127872-0-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\127872-0-0-0.wav
clip counter is: 114
resampled
output filename is 132073-1-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\132073-1-0-0.wav
clip counter is: 115
downmixed
resampled
output filename is 132073-1-1-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\132073-1-1-0.wav
clip counter is: 116
downmixed
resampled
output filename is 132073-1-2-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\132073-1-2-0.wav
clip counter is: 117
downmixed
resampled
output filename is 132073-1-3-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\132073-1-3-0.wav
clip counter is: 118
downmixed
resampled
output filename is 132073-1-4-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\

clip counter is: 169
downmixed
resampled
output filename is 147926-0-0-12-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-12.wav
clip counter is: 170
downmixed
resampled
output filename is 147926-0-0-16-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-16.wav
clip counter is: 171
downmixed
resampled
output filename is 147926-0-0-17-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-17.wav
clip counter is: 172
downmixed
resampled
output filename is 147926-0-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-2.wav
clip counter is: 173
downmixed
resampled
output filename is 147926-0-0-20-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-20.wav
clip counter is: 174
downmixed
resampled
output filename is 147926-0-0-21-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\147926-0-0-21.wav
clip counter is: 175
downmixed
resampled
output filename is 147926-0-0-27-mels

resampled
output filename is 155219-2-0-57-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\155219-2-0-57.wav
clip counter is: 229
downmixed
resampled
output filename is 155234-1-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\155234-1-0-0.wav
clip counter is: 230
downmixed
resampled
output filename is 156893-7-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\156893-7-0-0.wav
clip counter is: 231
downmixed
resampled
output filename is 156893-7-1-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\156893-7-1-0.wav
clip counter is: 232
downmixed
resampled
output filename is 156893-7-10-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\156893-7-10-0.wav
clip counter is: 233
downmixed
resampled
output filename is 156893-7-11-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold2\156893-7-11-0.wav
clip counter is: 234
downmixed
resampled
output filename is 156893-7-2-0-melspec-torch
N:\# GMU 2022 ML Model\Urb

...

downmixed
resampled
output filename is 28808-1-0-5-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-0-5.wav
clip counter is: 596
downmixed
resampled
output filename is 28808-1-0-6-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-0-6.wav
clip counter is: 597
downmixed
resampled
output filename is 28808-1-0-7-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-0-7.wav
clip counter is: 598
downmixed
resampled
output filename is 28808-1-0-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-0-8.wav
clip counter is: 599
downmixed
resampled
output filename is 28808-1-0-9-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-0-9.wav
clip counter is: 600
downmixed
resampled
output filename is 28808-1-1-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\28808-1-1-0.wav
clip counter is: 601
downmixed
resampled
output filename is 28808-1-1-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanS

downmixed
resampled
output filename is 71312-3-0-5-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\71312-3-0-5.wav
clip counter is: 653
downmixed
resampled
output filename is 72220-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\72220-3-0-0.wav
clip counter is: 654
downmixed
resampled
output filename is 72220-3-2-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\72220-3-2-2.wav
clip counter is: 655
downmixed
resampled
output filename is 72220-3-2-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\72220-3-2-3.wav
clip counter is: 656
downmixed
resampled
output filename is 72220-3-2-4-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\72220-3-2-4.wav
clip counter is: 657
downmixed
resampled
output filename is 72538-3-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\72538-3-0-0.wav
clip counter is: 658
downmixed
resampled
output filename is 72538-3-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanS

resampled
output filename is 74364-8-1-7-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74364-8-1-7.wav
clip counter is: 713
downmixed
resampled
output filename is 74364-8-1-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74364-8-1-8.wav
clip counter is: 714
downmixed
resampled
output filename is 74364-8-1-9-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74364-8-1-9.wav
clip counter is: 715
downmixed
resampled
output filename is 74922-4-0-0-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74922-4-0-0.wav
clip counter is: 716
downmixed
resampled
output filename is 74922-4-0-1-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74922-4-0-1.wav
clip counter is: 717
downmixed
resampled
output filename is 74922-4-0-2-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\74922-4-0-2.wav
clip counter is: 718
downmixed
resampled
output filename is 74922-4-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\aud

output filename is 83502-0-0-8-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\83502-0-0-8.wav
clip counter is: 770
downmixed
resampled
output filename is 83502-0-0-9-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\83502-0-0-9.wav
clip counter is: 771
downmixed
resampled
output filename is 84249-9-0-10-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\84249-9-0-10.wav
clip counter is: 772
downmixed
resampled
output filename is 84249-9-0-11-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\84249-9-0-11.wav
clip counter is: 773
downmixed
resampled
output filename is 84249-9-0-3-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\84249-9-0-3.wav
clip counter is: 774
downmixed
resampled
output filename is 84249-9-0-6-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\84249-9-0-6.wav
clip counter is: 775
downmixed
resampled
output filename is 84249-9-0-7-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fol

output filename is 99192-4-0-47-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-47.wav
clip counter is: 829
resampled
output filename is 99192-4-0-48-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-48.wav
clip counter is: 830
resampled
output filename is 99192-4-0-49-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-49.wav
clip counter is: 831
resampled
output filename is 99192-4-0-52-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-52.wav
clip counter is: 832
resampled
output filename is 99192-4-0-53-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-53.wav
clip counter is: 833
resampled
output filename is 99192-4-0-54-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-54.wav
clip counter is: 834
resampled
output filename is 99192-4-0-55-melspec-torch
N:\# GMU 2022 ML Model\UrbanSound8K\audio\fold10\99192-4-0-55.wav
clip counter is: 835
resampled